In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from fiona.crs import from_epsg

from glob import glob

import re, json

import seaborn as sns
sns.set_context("notebook", font_scale = 1.5)
sns.set_style("white", rc={"figure.figsize": (2.5, 0.5), "axes.linewidth" : 2})

from tqdm import tqdm

import warnings

%matplotlib inline

method_full = {"axis_ratio" : "Axis Ratio", "dist_a" : "Areal Distance", "dist_p" : "Population Distance",
               "dyn_radius" : "Dynamic Radius", "ehrenburg" : "Inscribed Circles", "exchange" : "Exchange", 
               "harm_radius" : "Harmonic Radius", "hull_a" : "Hull Area", "hull_p" : "Hull Population", 
               "inertia_a" : "Inertia Area", "inertia_p" : "Inertia Population", "mean_radius" : "Mean Radius", 
               "polsby" : "Isoperimeter Quotient", "polsby_w" : "County-Weighted IPQ",
               "power" : "Power Diagram", "reock" : "Circumscribing Circles", 
               "rohrbach" : "Distance to Perimeter", "split" : "Split-Line", "path_frac" : "Path Fraction",
               "107" : "107th Congress", "111" : "111th Congress", "114" : "114th Congress"}

## Measuring split counties.

### To replicate or not to replicate?
Change this flag to pull data from my private database instead of the prepared files.

In [2]:
REPLICATION = True

#### Here, I am caching for replication the tracts' intersecting "membership" in counties and _enacted_ Congressional Districts.  At the block level (block group for 1990), this is fairly "hefty": the database tables are several GB, and they have only point geometries.  So I am saving after the double group by statement (i.e., by county AND cd).

In [3]:
def cache_cd_sessn_split(usps, sessn):
    
    table = "census_blocks_2010"
    geom  = "geom"
    if int(sessn) == 111:
        table = "census_blocks_2000"
    if int(sessn) == 107:
        table = "census_bg_1990"
        geom  = "centroid"

    # Join points to CDs using within.
    # Cunties are already part of the GEOID hierarchy.
    cd = pd.read_sql("SELECT county, cd, SUM(pop) pop "
                     "FROM {} b "
                     "JOIN cd ON "
                     "  b.state = cd.state AND "
                     "  ST_Within(b.{}, cd.geom) "
                     "JOIN states s ON "
                     "  b.state = s.fips "
                     "WHERE pop > 0 AND usps = UPPER('{}') AND sessn = {} "
                     "GROUP BY county, cd "
                     "ORDER BY county, cd;".format(table, geom, usps, sessn), 
                     con = cen_con)
    
    # GROUP and SUM by county and cd.

    cd.to_csv("data/cd_splits/{}_{}.csv".format(usps, sessn), index = False)


if not REPLICATION:
    
    for usps in ["md", "nc", "pa"]:
        print(usps, "::", end = " ")
        for sessn in [107, 111, 114]:
            print(sessn, end = " ")
            cache_cd_sessn_split(usps, sessn)
        print()

#### Calculate the split fraction as defined in the paper: 

In [4]:
def cd_share(usps, sessn):

    cd = pd.read_csv("data/cd_splits/{}_{}.csv".format(usps, sessn))

    cd_sum        = cd.groupby("cd")            ["pop"].sum().reset_index().rename(columns = {"pop" : "cd_pop"})
    county_sum    = cd.groupby("county")        ["pop"].sum().reset_index().rename(columns = {"pop" : "county_pop"})
    cd_county_sum = cd.groupby(["cd", "county"])["pop"].sum().reset_index().rename(columns = {"pop" : "cd_county_pop"})

    splits = cd_county_sum.merge(cd_sum).merge(county_sum)
    splits["Xpop"] = splits[["county_pop", "cd_pop"]].min(axis = 1)
    splits["frac"] = splits["cd_county_pop"] / splits["Xpop"]

    s = (splits.frac * splits["cd_county_pop"]).sum() / splits["cd_county_pop"].sum()

    return [s]


#### Save the regionalizations for MD, PA, and NC to three compressed files.  

This comes of not anticipating this question by referees -- so the "replication" is a bit awkward.

The processed files are written to `data/splits/??_regions.csv.bz2`.

In [5]:
jdir = "data/c4_redux/"
s3_files = "/media/jsaxon/brobdingnag/data/s3/res/{}/final.csv"

polsby_w_json  = "/home/jsaxon/proj/cluscious/res/json/{}_polsby_w*.json"
polsby_w_files = "/home/jsaxon/proj/cluscious/res/{}/final.csv"

def get_state_files(s):

    files = {}
    
    print(s, end = " :: ")
    
    # Go through the json redux and load the corresponding csvs
    #   if it satisfies the population check.
    with open(jdir + "/{}_redux.json".format(s.lower())) as fi:
        for line in fi:

            j = json.loads(line)

            m = j["UID"].split("/")[1]

            if m == "axis_ratio":
                if j["PopulationDeviation"] > 0.05: continue
            elif j["PopulationDeviation"] > 0.02: continue

            if m not in files: files[m] = []

            files[m].append(s3_files.format(j["UID"]))

            
    # Now do the same thing for the weighted polsby files.
    files["polsby_w"] = []
    for line in list(glob(polsby_w_json.format(s))):
        
        with open(line) as json_file:
            j = json.load(json_file)
            if j["PopulationDeviation"] > 0.02: continue
            files["polsby_w"].append(polsby_w_files.format(j["UID"]))
            
    # Save all these as as single file.
    # THIS is what comes of not anticipating the regionalization.
    regionalization = []

    for m in files:
        print(m, end = " ")

        for f in files[m]:
            df = pd.read_csv(f, header = None, names = ["index", "cd"])

            df["method"] = m

            fmeta = f.split("/")
            df["uid"] = fmeta[-3] + "/" + fmeta[-2]

            regionalization.append(df)

    # Assemble the regionalizations from the individual CSVs/DFs.
    # Sort, reorder, and compress.
    regionalization = pd.concat(regionalization, sort = False).reset_index(drop = True)
    regionalization.sort_values(["method", "uid"], inplace = True)
    regionalization = regionalization[["method", "uid", "index", "cd"]]
    regionalization.to_csv("data/splits/{}_regions.csv.bz2".format(s), 
                           compression = "bz2", index = False)
                        
    print()
    return files


if not REPLICATION:
    
    get_state_files("md")
    get_state_files("nc")
    get_state_files("pa")

### Calculate the level of county splits for a dataframe.

This will be run as an "apply" function on a grouped dataframe.

In [6]:
tract_county_pop = pd.read_csv("data/tract_race.csv", usecols = ["usps", "rn", "cid", "pop"])
tract_county_pop.set_index("usps", inplace = True)
tract_county_pop.rename(columns = {"cid" : "county", "rn" : "index"}, inplace = True)

def split_rate(data, usps):
    
    # Get merge it with the county label and population.
    data = data.copy().merge(tract_county_pop.loc[usps.upper()])[["index", "cd", "county", "pop"]]

    # Group by county and CD, and then together,
    # to get the populations of the CD, county, and intersection.
    cd_sum        = data.groupby("cd")            ["pop"].sum().reset_index().rename(columns = {"pop" : "cd_pop"})
    county_sum    = data.groupby("county")        ["pop"].sum().reset_index().rename(columns = {"pop" : "county_pop"})
    cd_county_sum = data.groupby(["cd", "county"])["pop"].sum().reset_index().rename(columns = {"pop" : "cd_county_pop"})

    # The split fraction is the intersection pop. divided by 
    #   the lesser of the CD and the county pop.
    splits = cd_county_sum.merge(cd_sum).merge(county_sum)
    splits["Xpop"] = splits[["county_pop", "cd_pop"]].min(axis = 1)
    splits["frac"] = splits["cd_county_pop"] / splits["Xpop"]

    # This is the mean value, weighted by each fragment of the population.
    # That is, each individual intersection of a county and a Congressional district 
    # has a split fraction and a population.  The split fraction average
    # is weighted by the population, for the state-wide average.
    sval = (splits.frac * splits["cd_county_pop"]).sum() / splits["cd_county_pop"].sum()
    
    # print(splits)

    return sval

    shares = pd.Series([sval])
    shares.columns = ["split_frac"]
    return shares


### Potting Function

The plotting function here is nearly identical to the ones used in `main_plots.ipynb`.  The only real difference is the bounds, which are specified by the user.  Like competitiveness, this is a continuous variable...

In [7]:
size = (2.2, 0.5)
bins = np.arange(0, 1.001, 0.01)
def plot_common_county(data, state, method, vmin, vmax):

    # sns.set(rc={"figure.figsize": size, "axes.linewidth" : 4})
    f, ax = plt.subplots(1, sharex=True, sharey=True, figsize = size)


    if len(data) > 1:

        ## Warning about normed -> density keyword, 
        ##   from matplotlib through seaborn.
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            sns.distplot(data, ax = ax, bins = bins,
                         norm_hist = True, kde = False, 
                         hist_kws={"alpha" : 1.0, "color" : "#4DAFFF"})

    if len(data):

        mval = sum(data) / len(data)

        ## Warning about normed -> density keyword, 
        ##   from matplotlib through seaborn.
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            ax.plot([mval, mval], ax.get_ylim(), 
                    linewidth = 3, linestyle = "solid", 
                    c = "r" if method[0] == "1" else "k")


    sns.despine(left = True)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_yticks([])
    
    ax.set_xticks([0.6, 0.7, 0.8])
    ax.set_xlim(vmin, vmax)

    f.savefig("splits/{}_{}_ax.pdf".format(state, method), bbox_inches='tight', pad_inches=0)

    ax.set_xticks([])
    f.savefig("splits/{}_{}.pdf".format(state, method), bbox_inches='tight', pad_inches=0)

    plt.close("all")


In [8]:
vmin = {"md" : 0.53, "nc" : 0.65, "pa" : 0.55}
vmax = {"md" : 0.79, "nc" : 0.88, "pa" : 0.88}

communities_df = []
for s in ["md", "nc", "pa"]:
    
    print(s, end = "")
    communities = []
    # community_data = {}

    # Read in all of the data (!)
    state_regions = pd.read_csv("data/splits/{}_regions.csv.bz2".format(s))
    # state_regions.set_index("method", inplace = True)
    
    # Now groupby method and uid to get the average split value for each map.
    print(" :", end = "")
    sim_shares = state_regions.groupby(["method", "uid"]).apply(split_rate, usps = s)
    del state_regions
    print(": ", end = "")
    
    # Simulated plans
    for m in sorted(method_full):
    
        if "1" in m: continue
        print(m, end = " ")
            
        communities.append([m, sim_shares.loc[m].mean(),
                            "XZfigs/splits/{}_{}ZX".format(s.lower(), m)])
        
        # community_data[m] = list(shares)
        
    # for m, shares in community_data.items():
        plot_common_county(list(sim_shares.loc[m]), s, m, vmin[s], vmax[s])

    # Enacted plans
    for m in ["107", "111", "114"]:
    
        print(m, end = " ")
        shares = cd_share(s, m)
        communities.append([m, sum(shares) / len(shares),
                            "XZfigs/splits/{}_{}ZX".format(s.lower(), m)])
        plot_common_county(shares, s, m, vmin[s], vmax[s])
                

    print()

    cdf = pd.DataFrame(communities, columns = ["Method", "Mean", "Figure"])
    cdf.replace({"Method" : method_full}, inplace = True)
    cdf.set_index(["Method"], inplace = True)
        
    communities_df.append(cdf)
    
pd.options.display.precision = 2
cdf = pd.concat(communities_df, axis = 1)
cdf.columns = pd.MultiIndex.from_product([["Maryland", "North Carolina", "Pennsylvania"],
                                          ["Mean", "Figure"]])

cdf

md :: axis_ratio dist_a dist_p dyn_radius ehrenburg exchange harm_radius hull_a hull_p inertia_a inertia_p mean_radius path_frac polsby polsby_w power reock rohrbach split 107 111 114 
nc :: axis_ratio dist_a dist_p dyn_radius ehrenburg exchange harm_radius hull_a hull_p inertia_a inertia_p mean_radius path_frac polsby polsby_w power reock rohrbach split 107 111 114 
pa :: axis_ratio dist_a dist_p dyn_radius ehrenburg exchange harm_radius hull_a hull_p inertia_a inertia_p mean_radius path_frac polsby polsby_w power reock rohrbach split 107 111 114 


Maryland                                  \
                           Mean                          Figure   
Method                                                            
Axis Ratio                 0.67   XZfigs/splits/md_axis_ratioZX   
Areal Distance             0.70       XZfigs/splits/md_dist_aZX   
Population Distance        0.70       XZfigs/splits/md_dist_pZX   
Dynamic Radius             0.69   XZfigs/splits/md_dyn_radiusZX   
Inscribed Circles          0.67    XZfigs/splits/md_ehrenburgZX   
Exchange                   0.68     XZfigs/splits/md_exchangeZX   
Harmonic Radius            0.68  XZfigs/splits/md_harm_radiusZX   
Hull Area                  0.68       XZfigs/splits/md_hull_aZX   
Hull Population            0.66       XZfigs/splits/md_hull_pZX   
Inertia Area               0.69    XZfigs/splits/md_inertia_aZX   
Inertia Population         0.70    XZfigs/splits/md_inertia_pZX   
Mean Radius                0.69  XZfigs/splits/md_mean_radiusZX   
Path Fraction              0.62    XZfigs/splits/md_path_fracZX   
Isoperimeter Quotient      0.66       XZfigs/splits/md_polsbyZX   
County-Weighted IPQ        0.69     XZfigs/splits/md_polsby_wZX   
Power Diagram              0.69        XZfigs/splits/md_powerZX   
Circumscribing Circles     0.68        XZfigs/splits/md_reockZX   
Distance to Perimeter      0.67     XZfigs/splits/md_rohrbachZX   
Split-Line                 0.71        XZfigs/splits/md_splitZX   
107th Congress             0.71          XZfigs/splits/md_107ZX   
111th Congress             0.62          XZfigs/splits/md_111ZX   
114th Congress             0.58          XZfigs/splits/md_114ZX   

                       North Carolina                                  \
                                 Mean                          Figure   
Method                                                                  
Axis Ratio                       0.74   XZfigs/splits/nc_axis_ratioZX   
Areal Distance                   0.78       XZfigs/splits/nc_dist_aZX   
Population Distance              0.80       XZfigs/splits/nc_dist_pZX   
Dynamic Radius                   0.78   XZfigs/splits/nc_dyn_radiusZX   
Inscribed Circles                0.76    XZfigs/splits/nc_ehrenburgZX   
Exchange                         0.77     XZfigs/splits/nc_exchangeZX   
Harmonic Radius                  0.79  XZfigs/splits/nc_harm_radiusZX   
Hull Area                        0.76       XZfigs/splits/nc_hull_aZX   
Hull Population                  0.74       XZfigs/splits/nc_hull_pZX   
Inertia Area                     0.78    XZfigs/splits/nc_inertia_aZX   
Inertia Population               0.78    XZfigs/splits/nc_inertia_pZX   
Mean Radius                      0.78  XZfigs/splits/nc_mean_radiusZX   
Path Fraction                    0.73    XZfigs/splits/nc_path_fracZX   
Isoperimeter Quotient            0.77       XZfigs/splits/nc_polsbyZX   
County-Weighted IPQ              0.81     XZfigs/splits/nc_polsby_wZX   
Power Diagram                    0.78        XZfigs/splits/nc_powerZX   
Circumscribing Circles           0.77        XZfigs/splits/nc_reockZX   
Distance to Perimeter            0.75     XZfigs/splits/nc_rohrbachZX   
Split-Line                       0.77        XZfigs/splits/nc_splitZX   
107th Congress                   0.79          XZfigs/splits/nc_107ZX   
111th Congress                   0.73          XZfigs/splits/nc_111ZX   
114th Congress                   0.71          XZfigs/splits/nc_114ZX   

                       Pennsylvania                                  
                               Mean                          Figure  
Method                                                               
Axis Ratio                     0.69   XZfigs/splits/pa_axis_ratioZX  
Areal Distance                 0.70       XZfigs/splits/pa_dist_aZX  
Population Distance            0.73       XZfigs/splits/pa_dist_pZX  
Dynamic Radius                 0.71   XZfigs/splits/pa_dyn_radiusZX  
Inscribed Circles              

In [10]:
index_order = ['107th Congress', '111th Congress', '114th Congress', 
               'Areal Distance', 'Axis Ratio', 'Circumscribing Circles', 
               'Distance to Perimeter', 'Dynamic Radius', 'Exchange', 
               'Harmonic Radius', 'Hull Area', 'Hull Population', 
               'Inertia Area', 'Inertia Population', 'Inscribed Circles', 
               'Isoperimeter Quotient', 'County-Weighted IPQ', 
               'Mean Radius', 'Path Fraction', 'Population Distance', 
               'Power Diagram', 'Split-Line']

In [11]:
# cdf.sort_index(inplace = True)
cdf = cdf.loc[index_order]
tex = cdf.to_latex(na_rep = "", column_format = "l" + "c" * 6,
                                multicolumn_format = "c")

caption = """
Probability of a citizen of a county living in the same congressional district
  as another randomly-selected citizen of their county, for distributions of maps
  derived for various compactness definitions.
The ``probability" is modified to account for counties larger than
  Congressional Districts (see text).
North Carolina gained a seat after the 2000 Census, 
  whereas Pennsylvania lost seats in both 2002 and 2012.
The probabilities do not not correct for this effect (first three rows).
There is a progression towards more-dividied counties
  among the enacted maps.
For Maryland and North Carolina, the automated procedures 
  outperform the enacted maps.
The county-weighted isoperimeter quotient measure outperforms 
  baseline IPQ by a few percent.
"""


tex = re.sub(r" *(Maryland) \& *", r" \\multicolumn{2}{c}{ \\selectfont \1} ", tex)
tex = re.sub(r" *(North Carolina) \& *", r" \\multicolumn{2}{c}{ \\selectfont \1} ", tex)
tex = re.sub(r" *(Pennsylvania) \& *", r" \\multicolumn{2}{c}{ \\selectfont \1} ", tex)

tex = re.sub(r".*Mean.*\n", "", tex)
tex = re.sub(r".*Method.*\n", "", tex)

tex = tex.replace("splitZX", "split_axZX")

tex = tex.replace("XZ", "\includegraphics[width=6.5em]{")
tex = tex.replace("\_", "_")
tex = tex.replace("ZX", "}")

# tex = tex.replace("nan", "")
tex = re.sub("None", "\\\includegraphics[width=6.5em]{mini_hist/blank}", tex)

tex = tex.replace("toprule", "hline \\hline \\\\ ")
tex = tex.replace("\midrule", "\\\\ \\hline \\\\")
tex = tex.replace("\\bottomrule", "\hline \hline")

tex = tex + "\caption{" + caption + "}"
tex = tex + "\label{tab:competitiveness}"

tex = "\n\\begin{table}\n\\renewcommand{\\arraystretch}{0.7}\n " + tex + "\n\\end{table}\n "

tex = tex.replace("figs/", "")

tex = tex.replace("Areal Distance", "\\\\ \hline \\\\ \nAreal Distance")

tex = re.sub(r"(Split-Line)(.*)(0.[0-9]{2})(.*)(0.[0-9]{2})(.*)(0.[0-9]{2})", 
             r"\\raisebox{1.3em}{\1} \2 \\raisebox{1.3em}{\3} \4 \\raisebox{1.3em}{\5} \6 \\raisebox{1.3em}{\7} ", tex)

# tex = "^NT:\n\n" + tex

with open("tex/splits_table.tex", "w") as o: o.write(tex)